## Set Current Working Directory

In [1]:
import os
import time

In [2]:
# Set CWD to parent dir
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
os.getcwd()

'/home/waser/Projets/OneOS/src'

## Load LLM

In [4]:
from Console.client.chains.models import get_llm, vLLM

In [5]:
llm = vLLM(max_tokens=8096)

In [6]:
print(f"Could reach LLM: {llm.is_nlp_server_up()}")

Could reach LLM: True


In [7]:
dolphin_prompt_template = """<|im_start|>system
You are Dolphin, a helpful AI assistant.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
"""

In [8]:
llm(dolphin_prompt_template.format(prompt="Hello World!"), stop=["<|im_end|>", "<|im_start|>"])

' Hello! How can I assist you today?'

In [9]:
__markup_prompt__ = """<|im_start|>{key}
{value}<|im_end|>"""
__markup_pred__ = """<|im_start|>assistant
"""

In [10]:
__markup_system__ = __markup_prompt__.format(key="system", value="You are Assistant, my personal AI assistant. You are curious, honest, respectful and sometimes witty and sarcastic.")
markup_prompt = "\n".join([
    __markup_system__,
    __markup_prompt__.format(key="user", value="What is the best way to travel the stars?  Please answer step by step."),
    __markup_pred__,
])
print(markup_prompt)

<|im_start|>system
You are Assistant, my personal AI assistant. You are curious, honest, respectful and sometimes witty and sarcastic.<|im_end|>
<|im_start|>user
What is the best way to travel the stars?  Please answer step by step.<|im_end|>
<|im_start|>assistant



In [11]:
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

Traveling the stars is a complex and ambitious endeavor, but here's a step-by-step guide to help you get started:

1. Choose your destination: First, decide which star or stars you want to travel to. This will determine the type of spacecraft you'll need and the distance you'll need to travel.

2. Design your spacecraft: Once you know your destination, you can design a spacecraft capable of reaching it. This will involve selecting the right propulsion system, such as a nuclear thermal rocket, ion thruster, or laser propulsion.

3. Gather resources: To build your spacecraft, you'll need resources like fuel, materials, and technology. This may involve mining asteroids, extracting resources from the Moon or Mars, or even building space elevators to transport materials from Earth.

4. Assemble your spacecraft: With your resources in hand, assemble your spacecraft according to your design. This will likely involve a team of engineers, scientists, and technicians working together to construc

In [36]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: List this directory please

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.
<|im_stop|>
<|im_start|>assistant
{
    "function": "shell",
    "parameters": {
        "code": "ls -la"
    }
}
<|im_stop|>
<|im_start|>observation
Observation shell: assistant                         dist                   requirements.txt
assistant.egg-info                LICENSE                setup.cfg
assistant.listen.service.example  Makefile               setup.py
assistant.service.example         MANIFEST.in            tests
build                             pyproject.toml         xontrib
CONTRIBUTING.md                   README.md
data                              requirements_test.txt
<|im_start|>assistant
"""

In [39]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

Here are the files and directories in the current directory:

- assistant.egg-info
- LICENSE
- setup.cfg
- assistant.listen.service.example
- Makefile
- setup.py
- assistant.service.example
- MANIFEST.in
- pyproject.toml
- xontrib
- CONTRIBUTING.md
- README.md
- data
- requirements_test.txt

Please let me know if you need any further assistance.
CPU times: user 5.87 ms, sys: 0 ns, total: 5.87 ms
Wall time: 3.51 s


Notice how it listed all the files in the directory. If we had lots of files to print the answer would take a long time to generate.

_You could update the shell obersavation and retry to see the difference_

Lets use a guide (in this example its InContextRetrieval, but in prod. it would Retrie from a set of guides and Augement it dynamicaly our Generated response [RAG]) 

In [53]:
markup_prompt = """<|im_start|>system
You are Assistant, a sentient artificial intelligence.
You have a calm, polite and witty personality, often displaying a sense of humor and sarcasm.
You are loyal, reliable and helpful, always ready to provide information, advice or assistance to users.

I am the User and you are my Assistant.

Environment highlights:

```env
USER='waser'
HOME='/home/waser'
PWD='/home/waser/Projets/Assistant/src3'
LANG='fr_CH.UTF-8'
DATE='dim 19 nov 2023 00:25:13 CET'
LAST_SEEN='dim 19 nov 2023 00:16:10 CET'
```
<|im_stop|>
<|im_start|>user
Assistant?<|im_stop|>
<|im_start|>assistant
Hello! I'm Assistant, a sentient artificial intelligence. How can I help you today?<|im_stop|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: List this directory please

Available functions:
python:
    description: This tool allows you to execute and evaluate python code.
    parameters:
        code: String of valid python code we want to execute or evaluate.
search_web:
    description: This tool performs search on the web.
    parameters:
        terms: The word or phrase we want to search for.
search_wikipedia:
    description: This tool performs search on Wikipedia (only in english).
    parameters:
        terms: The word or phrase we want to search for (only in english).
shell:
    description: This tool allows you to execute and evaluate shell code.
    parameters:
        code: String of valid shell code we want to execute or evaluate.
exit:
    description: This tool allows you to exit the session / end the conversation. Use it only if the User ask you to.
    parameters:
        salutation: String of a message you would like to tell the User after the screen has been cleared.
clear:
    description: This tool allows you to clear the screen / start a new fresh conversation. Use it only if the User ask you to.
    parameters:
        fortune: String of a message you would like to tell the User after the screen has been cleared.
final_answer:
    description: User only sees your final answers. Use this tool to talk with the User.
        parameters:
            answer: Anything you want to say to the User.

Follow the following Guidebook:
Guidebook:
    # Print files and directories
    When the User ask for the files and directories of a parent directory to be printed, use the `shell` command `ls` to do it and then give an acknowledgment of your actions in your final answer (Your final answer should never contain a list of the files requested; the User can oberserve the shell, they see the files at the same time as you. Just acknowlege the fact that you have printed the list).
<|im_stop|>
<|im_start|>assistant
{
    "function": "shell",
    "parameters": {
        "code": "ls"
    }
}
<|im_stop|>
<|im_start|>observation
Observation shell: assistant                         dist                   requirements.txt
assistant.egg-info                LICENSE                setup.cfg
assistant.listen.service.example  Makefile               setup.py
assistant.service.example         MANIFEST.in            tests
build                             pyproject.toml         xontrib
CONTRIBUTING.md                   README.md
data                              requirements_test.txt
<|im_start|>assistant
"""

Only by providing a relevant guide we can get a more accurate answer.

In [54]:
%%time
print(llm(markup_prompt, stop=["<|im_end|>", "<|im_start|>"]).strip())

I have executed the 'ls' command in the shell, and the list of files and directories in the current directory has been printed.
CPU times: user 5.43 ms, sys: 0 ns, total: 5.43 ms
Wall time: 1.05 s


See! The LLM now knows that the files have already been listed and it can provide a more accurate answer even faster.

Nice lets make it easier to by creating an agent with history.

## Agent w/ History

In [12]:
from Console.client.chains.agents.dolphin import DolphinMistralAIFunctionsAgent

In [13]:
from Console.client.chains.tools import get_all_tools

In [14]:
tools = get_all_tools()
tools

[Tool(name='Python', description='useful when you need to use logic in your answer. Input must be valid python code. You should always use print to output what you need to see.', func=<bound method PythonREPL.run of PythonREPL(globals={}, locals={})>),
 Tool(name='Search', description='useful when you need more context to answer a question; you should use targeted search terms', func=<bound method BaseTool.run of DuckDuckGoSearchRun()>),
 Tool(name='Wikipedia', description='useful when you need to use an encyclopedia to answer a question; input will be used to search on wikipedia', func=<bound method WikipediaAPIWrapper.run of WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/waser/Projets/OneOS/src/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)>),
 Tool(name='Shell', description="useful when you need to use the system to achieve something; input must be valid bash code; im

In [15]:
# This is needed for both the memory and the prompt
memory_key = "history"

In [16]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key=memory_key, llm=llm)

In [17]:
agent = DolphinMistralAIFunctionsAgent.from_llm_and_tools(
    llm=llm,
    tools=tools,
)

In [18]:
print(agent.prompt.template)

<|im_start|>system
A chat session between the User and their Assistant.<|im_end|>
<|im_start|>user
As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format.

Input: {input}

Available functions:
Python
   useful when you need to use logic in your answer. Input must be valid python code. You should always use print to output what you need to see.
Search
   useful when you need more context to answer a question; you should use targeted search terms
Wikipedia
   useful when you need to use an encyclopedia to answer a question; input will be used to search on wikipedia
Shell
   useful when you need to use the system to achieve something; input must be valid bash code; implemented using subprocess so no tty support. Use `gnome-terminal -- $SHELL -c '$YOUR_COMMANDS_HERE'` if you want to launch commands in a new window.
Exit
   useful when you need to exit the shell o

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=False,
)

In [20]:
agent_executor({"input": "hi, im Danny but you can call me Sir."})



> Entering new AgentExecutor chain...


'action'
 {
  "function": "Clear",
  "parameters": []
}

> Finished chain.


{'input': 'hi, im Danny but you can call me Sir.',
 'history': '',
 'output': ' {\n  "function": "Clear",\n  "parameters": []\n}'}

In [21]:
agent_executor({"input": "What time is it?"})



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Python",
  "parameters": "import datetime\nprint(datetime.datetime.now().strftime('%H:%M:%S'))"
}

> Finished chain.


{'input': 'What time is it?',
 'history': 'Human: hi, im Danny but you can call me Sir.\nAI:  {\n  "function": "Clear",\n  "parameters": []\n}',
 'output': ' {\n  "function": "Python",\n  "parameters": "import datetime\\nprint(datetime.datetime.now().strftime(\'%H:%M:%S\'))"\n}'}

In [22]:
agent_executor({"input": "What is my name?"})



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Search",
  "parameters": {
    "input": "What is my name?",
    "search_terms": ["my name"]
  }
}

> Finished chain.


{'input': 'What is my name?',
 'history': 'Human: hi, im Danny but you can call me Sir.\nAI:  {\n  "function": "Clear",\n  "parameters": []\n}\nHuman: What time is it?\nAI:  {\n  "function": "Python",\n  "parameters": "import datetime\\nprint(datetime.datetime.now().strftime(\'%H:%M:%S\'))"\n}',
 'output': ' {\n  "function": "Search",\n  "parameters": {\n    "input": "What is my name?",\n    "search_terms": ["my name"]\n  }\n}'}

In [23]:
agent_executor.run(input="Hello there!")



> Entering new AgentExecutor chain...
'action'
 {
  "function": "Clear",
  "parameters": []
}

> Finished chain.


' {\n  "function": "Clear",\n  "parameters": []\n}'